In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
diabetes = pd.read_csv("diabetes.csv")
df = diabetes.copy()
df = df.dropna()
y = df["Outcome"]
X = df.drop(['Outcome'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

ysa'larının değişkenlerin ölçeklerine ve bunların birbirleri aralarındaki durumlardan etkilenir ve bundan dolayı standartlaştırma işlemi gerekir.

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler = StandardScaler()

In [5]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
from sklearn.neural_network import MLPClassifier

In [8]:
mlpc = MLPClassifier().fit(X_train_scaled,y_train)
#oncelikle hic arguman girmeden modeli kuruyoruz.

In [9]:
mlpc.coefs_

[array([[ 5.38398736e-02, -3.90603566e-01, -5.79692807e-03,
         -1.43335506e-01, -1.79863730e-01, -8.90427327e-02,
         -2.33767506e-01, -3.77235080e-02, -8.63908350e-02,
          6.83649699e-02, -2.56631287e-01, -4.95084502e-02,
         -3.21089651e-01,  6.40136996e-02, -9.06979220e-02,
         -1.92954573e-01, -1.94523724e-01, -1.75996400e-01,
         -2.69599066e-01, -8.69757795e-02, -1.97878759e-01,
          1.12837673e-01, -3.25797137e-01, -4.59995987e-02,
         -8.97500909e-02, -1.97282244e-01,  2.28075560e-01,
         -4.02116149e-01, -3.88874548e-01, -2.11609819e-01,
          2.59300448e-01, -7.23607446e-02,  7.15955659e-02,
          8.70778596e-02, -1.82810027e-02, -3.52723832e-01,
         -8.28706451e-02,  3.78476150e-02, -2.59935278e-01,
          3.31677478e-01, -1.76879798e-01,  2.03998400e-01,
         -2.59881929e-01, -1.22825602e-02, -4.69439187e-02,
         -4.15172593e-01,  2.91837269e-01, -3.17846379e-01,
         -1.71781732e-01,  1.78123137e-0

In [11]:
# Model nesnelerine erismek icin
dir(mlpc)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_backprop',
 '_check_feature_names',
 '_check_n_features',
 '_check_solver',
 '_compute_loss_grad',
 '_estimator_type',
 '_fit',
 '_fit_lbfgs',
 '_fit_stochastic',
 '_forward_pass',
 '_forward_pass_fast',
 '_get_param_names',
 '_get_tags',
 '_init_coef',
 '_initialize',
 '_label_binarizer',
 '_loss_grad_lbfgs',
 '_more_tags',
 '_no_improvement_count',
 '_optimizer',
 '_random_state',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_unpack',
 '_update_no_improvement_count',
 '_validate_data',
 '_validate_hyperparameters',
 '_validate_inpu

In [12]:
y_pred = mlpc.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.7316017316017316

### Model Tuning

In [14]:
?mlpc

In [16]:
mlpc_params = {"alpha": [0.1, 0.01, 0.02, 0.005, 0.0001,0.00001],
              "hidden_layer_sizes": [(10,10,10),
                                     (100,100,100),
                                     (100,100),
                                     (3,5), 
                                     (5, 3)],
              "solver" : ["lbfgs","adam","sgd"],
              "activation": ["relu","logistic"]}

#### Grid Search Yontemi

In [17]:
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, 
                         cv = 10, 
                         n_jobs = -1,
                         verbose = 2)

mlpc_cv_model.fit(X_train_scaled, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


GridSearchCV(cv=10, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005, 0.0001, 1e-05],
                         'hidden_layer_sizes': [(10, 10, 10), (100, 100, 100),
                                                (100, 100), (3, 5), (5, 3)],
                         'solver': ['lbfgs', 'adam', 'sgd']},
             verbose=2)

In [18]:
print("En iyi parametreler: " + str(mlpc_cv_model.best_params_))

En iyi parametreler: {'activation': 'relu', 'alpha': 0.02, 'hidden_layer_sizes': (100, 100), 'solver': 'sgd'}


In [19]:
mlpc_tuned = MLPClassifier(activation = "relu", 
                           alpha = 0.02, 
                           hidden_layer_sizes = (100, 100),
                          solver = "sgd")

In [20]:
mlpc_tuned.fit(X_train_scaled, y_train)

MLPClassifier(alpha=0.02, hidden_layer_sizes=(100, 100), solver='sgd')

In [21]:
y_pred = mlpc_tuned.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7489177489177489